# Part I:
## 1.A. Scratch code & testing
Below is the code as I was writing it. Section 1.B. contains the entire code in a single cell with user input (section 1.A is only testing)

In [328]:
import numpy as np
import pandas as pd
import os
import re
from statistics import mean
from itertools import islice

In [270]:
#FASTA file parser code

file="FinalProject.fasta"

f=open(file)
sequence_names=[]
sequences=[]
count=0
seq=""

for line in f:
    if line.startswith(">"):
        sequence_names.append(line[1:].strip()) #to remove the > and \n
        count=count+1
        sequences.append("") #initializing the slot for the sequence
    elif not line.startswith('#') and line!="":
        sequences[count-1]=sequences[count-1]+line.strip().replace('-',"")

f.close()

#initialize data
data = {'ID':sequence_names,
        'Sequence':sequences}
  
#dataframe
dfseq = pd.DataFrame(data)

#print
dfseq
dfseq=dfseq.set_index('ID')

In [271]:
#number of sequences in file
len(dfseq)

6

In [272]:
#extracting primers, introns, and exons, and storing then in pandas dataframe
primers=[]
exons=[]
introns=[]

for i in range(0, len(dfseq)):
    p=re.findall('[a-z]+', dfseq.Sequence[i])[0] #the first lowercase is the primer
    primers.append(p)
    
    ex=re.findall('[A-Z]+', dfseq.Sequence[i]) #all uppercases are exons
    exons.append(ex)
    
    intr=re.findall('[a-z]+', dfseq.Sequence[i])[1:-1] #all lowercases are introns: exclude upperstream and lowerstream
    introns.append(intr)
    
#adding primer, exons and introns to the dataframe
dfseq['Primer']=primers
dfseq['Exons']=exons
dfseq['Introns']=introns

#dataframe with seq ID and all its corresponding information
dfseq

,Sequence,Primer,Exons,Introns
ID,,,,
HCN1,gccgaggggaggcgctggggcgcgagggctGAGCCGAGCTGAGCCT...,gccgaggggaggcgctggggcgcgagggct,[GAGCCGAGCTGAGCCTCGCCTAGCTCCGGCAGCCTCAGCTTCAGC...,[gtgagggctcccggtcgccgccccaccctcccttcaggcgcgccc...
HCN2,ccgccccgccccgcccccgcctcccccctCCCTCGGGCTCCGGCCG...,ccgccccgccccgcccccgcctcccccct,[CCCTCGGGCTCCGGCCGGCGGCGGCGGCGGCGGCTCCGCTCCGCA...,[gtaccgcctccgggagggccggtcggcgcgagggggcccggggag...
KCNA1,gggctgcgcttcgagacgcagctcaagaccCTGGCGCAGTTCCCCA...,gggctgcgcttcgagacgcagctcaagacc,[CTGGCGCAGTTCCCCAACACGCTGCTGGGCAACCCTAAGAAACGC...,[gtaagcatgtttgaatctgatacaatttattttataatcgcatgc...
KCNA2,agcgctgaccaagaaaggaagtggtgatggGGCACATAGAAGAGTG...,agcgctgaccaagaaaggaagtggtgatgg,[GGCACATAGAAGAGTGAGCCATCATCTGGTTTCCAGCGCCAAGAC...,[gtgagagaagaggaggggcagcaagggtgagcatgcatgtgtgga...
KCNS3,cctcgctctagcggggcgggaccgacggacAGACCGGCCGACGCGG...,cctcgctctagcggggcgggaccgacggac,[AGACCGGCCGACGCGGGCCACCCCGCTCTCCTCGCCGCCGCGGCG...,[gtacctggctggcgtccccacctccctgcgcgctccggagacttc...
KIR3,gcgcggccgcctgtctgcaccggcagcaccATGTCGCTCATGGTCG...,gcgcggccgcctgtctgcaccggcagcacc,"[ATGTCGCTCATGGTCGTCAGCATGGCGTGTGTTG, GAGTCCACA...",[gtgagtcctggaaaggaatagagggagggagtgccacatcctcct...


In [273]:
dfseq

,Sequence,Primer,Exons,Introns
ID,,,,
HCN1,gccgaggggaggcgctggggcgcgagggctGAGCCGAGCTGAGCCT...,gccgaggggaggcgctggggcgcgagggct,[GAGCCGAGCTGAGCCTCGCCTAGCTCCGGCAGCCTCAGCTTCAGC...,[gtgagggctcccggtcgccgccccaccctcccttcaggcgcgccc...
HCN2,ccgccccgccccgcccccgcctcccccctCCCTCGGGCTCCGGCCG...,ccgccccgccccgcccccgcctcccccct,[CCCTCGGGCTCCGGCCGGCGGCGGCGGCGGCGGCTCCGCTCCGCA...,[gtaccgcctccgggagggccggtcggcgcgagggggcccggggag...
KCNA1,gggctgcgcttcgagacgcagctcaagaccCTGGCGCAGTTCCCCA...,gggctgcgcttcgagacgcagctcaagacc,[CTGGCGCAGTTCCCCAACACGCTGCTGGGCAACCCTAAGAAACGC...,[gtaagcatgtttgaatctgatacaatttattttataatcgcatgc...
KCNA2,agcgctgaccaagaaaggaagtggtgatggGGCACATAGAAGAGTG...,agcgctgaccaagaaaggaagtggtgatgg,[GGCACATAGAAGAGTGAGCCATCATCTGGTTTCCAGCGCCAAGAC...,[gtgagagaagaggaggggcagcaagggtgagcatgcatgtgtgga...
KCNS3,cctcgctctagcggggcgggaccgacggacAGACCGGCCGACGCGG...,cctcgctctagcggggcgggaccgacggac,[AGACCGGCCGACGCGGGCCACCCCGCTCTCCTCGCCGCCGCGGCG...,[gtacctggctggcgtccccacctccctgcgcgctccggagacttc...
KIR3,gcgcggccgcctgtctgcaccggcagcaccATGTCGCTCATGGTCG...,gcgcggccgcctgtctgcaccggcagcacc,"[ATGTCGCTCATGGTCGTCAGCATGGCGTGTGTTG, GAGTCCACA...",[gtgagtcctggaaaggaatagagggagggagtgccacatcctcct...


In [ ]:
#FUNCTIONS (to be used with dataframe.apply)

#function that returns average string length in a list of strings
#uses mean from statistics package

def avgstrlen(list):
    return mean([len(i) for i in list])

#function that calculates the percentage occurence of a letter in a list of strings

def occurencepercentage(letter, list):
    str=''.join(list).upper() #case insensitive, join list elements into a single string
    freq=str.count(letter.upper()) #occurence of letter in string
    perc=100*(float(freq)/float(len(str))) #percentage
    return perc

#functions that calculate percentage occurence of each nucleotide (to be used with apply)
def occurenceA(list):
    return occurencepercentage('A', list)

def occurenceG(list):
    return occurencepercentage('G', list)

def occurenceC(list):
    return occurencepercentage('C', list)

def occurenceT(list):
    return occurencepercentage('T', list)

In [220]:
#number of introns
dfseq['Introns'].apply(len)

ID
HCN1     7
HCN2     7
KCNA1    1
KCNA2    4
KCNS3    2
KIR3     2
Name: Introns, dtype: int64

In [199]:
#number of exons
dfseq['Exons'].apply(len)

ID
HCN1     8
HCN2     8
KCNA1    2
KCNA2    5
KCNS3    3
KIR3     3
Name: Exons, dtype: int64

In [201]:
#average intron length per sequence
dfseq['Introns'].apply(avgstrlen)

ID
HCN1     61642.857143
HCN2      3408.428571
KCNA1     4062.000000
KCNA2     8984.000000
KCNS3    25975.500000
KIR3      1668.000000
Name: Introns, dtype: float64

In [202]:
#average exon length per sequence
dfseq['Exons'].apply(avgstrlen)

ID
HCN1     1241.625000
HCN2      427.500000
KCNA1    1309.000000
KCNA2     391.800000
KCNS3     780.333333
KIR3      209.333333
Name: Exons, dtype: float64

In [207]:
d=dfseq['Introns'].apply(occurenceC)
d

ID
HCN1     16.936732
HCN2     28.710340
KCNA1    20.827179
KCNA2    23.216273
KCNS3    19.254682
KIR3     20.233813
Name: Introns, dtype: float64

In [208]:
d[0]

16.936732329084588

In [226]:
#sequence IDS
list(dfseq.index)

['HCN1', 'HCN2', 'KCNA1', 'KCNA2', 'KCNS3', 'KIR3']

In [232]:
#number of exons
dfseq['NumExons']=dfseq['Exons'].apply(len).tolist()

#number of introns
dfseq['NumIntrons']=dfseq['Introns'].apply(len).tolist()

#AvgExonLength
dfseq['AvgExonLength']=dfseq['Exons'].apply(avgstrlen).tolist()
#AvgIntronLength
dfseq['AvgIntronLength']=dfseq['Introns'].apply(avgstrlen).tolist()

#%A_Exon
dfseq['%A_Exon']=dfseq['Exons'].apply(occurenceA).tolist()
#%C_Exon
dfseq['%C_Exon']=dfseq['Exons'].apply(occurenceC).tolist()
#%G_Exon
dfseq['%G_Exon']=dfseq['Exons'].apply(occurenceG).tolist()
#%T_Exon
dfseq['%T_Exon']=dfseq['Exons'].apply(occurenceT).tolist()

#%A_Intron
dfseq['%A_Intron']=dfseq['Introns'].apply(occurenceA).tolist()
#%C_Intron
dfseq['%C_Intron']=dfseq['Introns'].apply(occurenceC).tolist()
#%G_Intron
dfseq['%G_Intron']=dfseq['Introns'].apply(occurenceG).tolist()
#%T_Intron
dfseq['%T_Intron']=dfseq['Introns'].apply(occurenceT).tolist()

In [233]:
dfseq

,Sequence,Primer,Exons,Introns,NumExons,NumIntrons,AvgExonLength,AvgIntronLength,%A_Exon,%C_Exon,%G_Exon,%T_Exon,%A_Intron,%C_Intron,%G_Intron,%T_Intron
ID,,,,,,,,,,,,,,,,
HCN1,gccgaggggaggcgctggggcgcgagggctGAGCCGAGCTGAGCCT...,gccgaggggaggcgctggggcgcgagggct,[GAGCCGAGCTGAGCCTCGCCTAGCTCCGGCAGCCTCAGCTTCAGC...,[gtgagggctcccggtcgccgccccaccctcccttcaggcgcgccc...,8,7,1241.625000,61642.857143,30.252693,20.215443,20.004027,29.527837,32.098494,16.936732,17.690151,33.274623
HCN2,ccgccccgccccgcccccgcctcccccctCCCTCGGGCTCCGGCCG...,ccgccccgccccgcccccgcctcccccct,[CCCTCGGGCTCCGGCCGGCGGCGGCGGCGGCGGCTCCGCTCCGCA...,[gtaccgcctccgggagggccggtcggcgcgagggggcccggggag...,8,7,427.500000,3408.428571,15.116959,38.713450,32.105263,14.064327,17.527977,28.710340,33.345907,20.415776
KCNA1,gggctgcgcttcgagacgcagctcaagaccCTGGCGCAGTTCCCCA...,gggctgcgcttcgagacgcagctcaagacc,[CTGGCGCAGTTCCCCAACACGCTGCTGGGCAACCCTAAGAAACGC...,[gtaagcatgtttgaatctgatacaatttattttataatcgcatgc...,2,1,1309.000000,4062.000000,27.731092,24.407945,21.275783,26.585180,28.385032,20.827179,20.704087,30.083703
KCNA2,agcgctgaccaagaaaggaagtggtgatggGGCACATAGAAGAGTG...,agcgctgaccaagaaaggaagtggtgatgg,[GGCACATAGAAGAGTGAGCCATCATCTGGTTTCCAGCGCCAAGAC...,[gtgagagaagaggaggggcagcaagggtgagcatgcatgtgtgga...,5,4,391.800000,8984.000000,24.400204,26.901480,24.196018,24.502297,25.486977,23.216273,23.413847,27.882903
KCNS3,cctcgctctagcggggcgggaccgacggacAGACCGGCCGACGCGG...,cctcgctctagcggggcgggaccgacggac,[AGACCGGCCGACGCGGGCCACCCCGCTCTCCTCGCCGCCGCGGCG...,[gtacctggctggcgtccccacctccctgcgcgctccggagacttc...,3,2,780.333333,25975.500000,23.878684,23.964118,25.544639,26.612559,25.373910,19.254682,21.911032,33.460376
KIR3,gcgcggccgcctgtctgcaccggcagcaccATGTCGCTCATGGTCG...,gcgcggccgcctgtctgcaccggcagcacc,"[ATGTCGCTCATGGTCGTCAGCATGGCGTGTGTTG, GAGTCCACA...",[gtgagtcctggaaaggaatagagggagggagtgccacatcctcct...,3,2,209.333333,1668.000000,20.541401,30.095541,25.000000,24.363057,32.224221,20.233813,27.607914,19.934053


In [235]:
#extracting only needed columns
dfseqstat=dfseq[['NumExons','NumIntrons', 'AvgExonLength', 'AvgIntronLength', '%A_Exon', '%C_Exon', '%G_Exon', '%T_Exon', '%A_Intron', '%C_Intron','%G_Intron','%T_Intron']]

In [245]:
#rounding for cleaner output
dfseqstat=dfseqstat.round()

In [250]:
#Write to DNAStats.txt
filename=file+'DNAStats.txt' #to avoid overwriting for multiple runs but with different filenames
dfseqstat.to_csv(filename, header=True, index=True, sep='\t')

In [251]:
print(dfseqstat)

       NumExons  NumIntrons  AvgExonLength  AvgIntronLength  %A_Exon  %C_Exon  \
ID                                                                              
HCN1          8           7         1242.0          61643.0     30.0     20.0   
HCN2          8           7          428.0           3408.0     15.0     39.0   
KCNA1         2           1         1309.0           4062.0     28.0     24.0   
KCNA2         5           4          392.0           8984.0     24.0     27.0   
KCNS3         3           2          780.0          25976.0     24.0     24.0   
KIR3          3           2          209.0           1668.0     21.0     30.0   

       %G_Exon  %T_Exon  %A_Intron  %C_Intron  %G_Intron  %T_Intron  
ID                                                                   
HCN1      20.0     30.0       32.0       17.0       18.0       33.0  
HCN2      32.0     14.0       18.0       29.0       33.0       20.0  
KCNA1     21.0     27.0       28.0       21.0       21.0       30.0  
K

In [258]:
print("The number of sequences is", len(dfseq))

The number of sequences is 6


In [279]:
#Retrieving sequence stats from created file

#would normally use the created df to output info given sequence ID, but asked to read created file, so:
newdf=pd.read_csv(filename, sep='\t', engine='python')
newdf=newdf.set_index('ID')
newdf

,NumExons,NumIntrons,AvgExonLength,AvgIntronLength,%A_Exon,%C_Exon,%G_Exon,%T_Exon,%A_Intron,%C_Intron,%G_Intron,%T_Intron
ID,,,,,,,,,,,,
HCN1,8,7,1242.0,61643.0,30.0,20.0,20.0,30.0,32.0,17.0,18.0,33.0
HCN2,8,7,428.0,3408.0,15.0,39.0,32.0,14.0,18.0,29.0,33.0,20.0
KCNA1,2,1,1309.0,4062.0,28.0,24.0,21.0,27.0,28.0,21.0,21.0,30.0
KCNA2,5,4,392.0,8984.0,24.0,27.0,24.0,25.0,25.0,23.0,23.0,28.0
KCNS3,3,2,780.0,25976.0,24.0,24.0,26.0,27.0,25.0,19.0,22.0,33.0
KIR3,3,2,209.0,1668.0,21.0,30.0,25.0,24.0,32.0,20.0,28.0,20.0


In [291]:
#example selection
selection="KCNA1"
info = newdf.loc[selection]
print(info)

NumExons              2.0
NumIntrons            1.0
AvgExonLength      1309.0
AvgIntronLength    4062.0
%A_Exon              28.0
%C_Exon              24.0
%G_Exon              21.0
%T_Exon              27.0
%A_Intron            28.0
%C_Intron            21.0
%G_Intron            21.0
%T_Intron            30.0
Name: KCNA1, dtype: float64


In [362]:
idx=newdf.index

'erhrw' not in idx

True

In [318]:
#Translation of selection
#getting the exon sequence of the user selection (SeqID)
exons=dfseq.loc[selection,'Exons']

#joining the exons into one exon string
exonseq=''.join(exons).upper()
exonseq=exonseq.replace("T", "U")

In [320]:
exonseq

'CUGGCGCAGUUCCCCAACACGCUGCUGGGCAACCCUAAGAAACGCAUGCGCUACUUCGACCCCCUGAGGAACGAGUACUUCUUCGACCGCAACCGGCCCAGCUUCGACGCCAUCCUCUACUACUACCAGUCCGGCGGCCGCCUGCGGAGGCCGGUCAACGUGCCCCUGGACAUGUUCUCCGAGGAGAUCAAGUUUUACGAGUUGGGCGAGGAGGCCAUGGAGAAGUUCCGGGAGGACGAGGGCUUCAUCAAGGAGGAGGAGCGCCCUCUGCCCGAGAAGGAGUACCAGCGCCAGGUGUGGCUGCUCUUCGAGUACCCCGAGAGCUCGGGGCCCGCCAGGGUCAUCGCCAUCGUCUCCGUCAUGGUCAUCCUCAUCUCCAUCGUCAUCUUUUGCCUGGAGACGCUCCCCGAGCUGAAGGAUGACAAGGACUUCACGGGCACCGUCCACCGCAUCGACAACACCACGGUCAUCUACAAUUCCAACAUCUUCACAGACCCCUUCUUCAUCGUGGAAACGCUGUGUAUCAUCUGGUUCUCCUUCGAGCUGGUGGUGCGCUUCUUCGCCUGCCCCAGCAAGACGGACUUCUUCAAAAACAUCAUGAACUUCAUAGACAUUGUGGCCAUCAUUCCUUAUUUCAUCACGCUGGGCACCGAGAUAGCUGAGCAGGAAGGAAACCAGAAGGGCGAGCAGGCCACCUCCCUGGCCAUCCUCAGGGUCAUCCGCUUGGUAAGGGUUUUUAGAAUCUUCAAGCUCUCCCGCCACUCUAAGGGCCUCCAGAUCCUGGGCCAGACCCUCAAAGCUAGUAUGAGAGAGCUAGGGCUGCUCAUCUUUUUCCUCUUCAUCGGGGUCAUCCUGUUUUCUAGUGCAGUGUACUUUGCCGAGGCGGAAGAAGCUGAGUCGCACUUCUCCAGUAUCCCCGAUGCUUUCUGGUGGGCGGUGGUGUCCAUGACCACUGUAGGAUACGGUGACAUGUACCCUGUGACAAUUGGAGGCAAGAUC

In [326]:
#function that translates a given mRNA into a protein sequence
def translate(seq):
    protein =""
    
    cmap = {"UUU":"F", "UUC":"F", "UUA":"L", "UUG":"L", "UCU":"S", "UCC":"S", "UCA":"S", 
           "UCG":"S", "UAU":"Y", "UAC":"Y", "UAA":"STOP", "UAG":"STOP", "UGU":"C", "UGC":"C", 
           "UGA":"STOP", "UGG":"W", "CUU":"L", "CUC":"L", "CUA":"L", "CUG":"L", "CCU":"P", 
           "CCC":"P", "CCA":"P", "CCG":"P", "CAU":"H", "CAC":"H", "CAA":"Q", "CAG":"Q", 
           "CGU":"R", "CGC":"R", "CGA":"R", "CGG":"R", "AUU":"I", "AUC":"I", "AUA":"I", 
           "AUG":"M", "ACU":"T", "ACC":"T", "ACA":"T", "ACG":"T", "AAU":"N", "AAC":"N", 
           "AAA":"K", "AAG":"K", "AGU":"S", "AGC":"S", "AGA":"R", "AGG":"R", "GUU":"V", 
           "GUC":"V", "GUA":"V", "GUG":"V", "GCU":"A", "GCC":"A", "GCA":"A", "GCG":"A", 
           "GAU":"D", "GAC":"D", "GAA":"E", "GAG":"E", "GGU":"G", "GGC":"G", "GGA":"G", "GGG":"G"}

    if len(seq)%3 != 0: #if the sequence isn't divisible by 3
        skip=len(seq)%3 #skip the remainder (either last 1 or 2 bases)
    
    for i in range(0, len(seq)-skip, 3):
        codon = seq[i:i+3]
        protein=protein+cmap[codon]
    
    return protein

translate(exonseq)

'LAQFPNTLLGNPKKRMRYFDPLRNEYFFDRNRPSFDAILYYYQSGGRLRRPVNVPLDMFSEEIKFYELGEEAMEKFREDEGFIKEEERPLPEKEYQRQVWLLFEYPESSGPARVIAIVSVMVILISIVIFCLETLPELKDDKDFTGTVHRIDNTTVIYNSNIFTDPFFIVETLCIIWFSFELVVRFFACPSKTDFFKNIMNFIDIVAIIPYFITLGTEIAEQEGNQKGEQATSLAILRVIRLVRVFRIFKLSRHSKGLQILGQTLKASMRELGLLIFFLFIGVILFSSAVYFAEAEEAESHFSSIPDAFWWAVVSMTTVGYGDMYPVTIGGKIVGSLCAIAGVLTIALPVPVIVSNFNYFYHRETEGEEQAQLLHVSSPNLASDSDLSRRSSSTMSKSEYMEIEEDMNNSIAHYRQVNIRTANCTTANQNCVNKSKLLTDVSTOPKTKASKQKSPTSTOPQLKRLKKQNRKPSDSCHALSTOPILYSTOPVDLECSISTOPLSMRCCIEDFGGGEPEAFKIHDKINYFPFIKKWEKREYFLKLASTOPKDSVHELVSTOPVKSTOPSTOPSYASPNSTOPNIFNALVSLTFLKTQNKMITSTOPKYEIEIRMGLQSTOPNIFANCVAHSTOPRQCIRCIICNMIDQPKWTMNRYFYFDQLNCILQGEKRKLRLLKTGSQSTLSTOPIGYWSSTOPSVGISPWAHSLSNPDYSLRKSSTOPLNSTOPINSTOPFFCSAAKWSQLQMSQIQVFSHQACTPTPGFQNWMSTOPNLSLLIAREHKSTOPSSTOPMSKLSPLQTLQKHLSSLQRTVLGLFMQQMSTOPVDNMDSMSTOPHASNSKDKVLLRLKIKIESTOPYSTOPFKVHHQDNKPFKLKKRYFISSTOPVCQLLPPSTOPVKDVSHLHLLRILPSLTVSTOPYVRSRTLNVMNSTOPEPNSTOPCASTOPFSSMQFYLLLSLYDLFIFG'

In [336]:
#primer melting temperature: range & average

#getting the primer sequence of the user selection (SeqID)
primer=dfseq.loc[selection,'Primer']
primer

'gggctgcgcttcgagacgcagctcaagacc'

In [332]:
#uses islice
#function that returns a list of sliding window strings
def window(seq, n=20):
    #sliding window of seq of width n
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

primerwindows=["".join(x) for x in window(primer, 20)]
primerwindows

['gggctgcgcttcgagacgca',
 'ggctgcgcttcgagacgcag',
 'gctgcgcttcgagacgcagc',
 'ctgcgcttcgagacgcagct',
 'tgcgcttcgagacgcagctc',
 'gcgcttcgagacgcagctca',
 'cgcttcgagacgcagctcaa',
 'gcttcgagacgcagctcaag',
 'cttcgagacgcagctcaaga',
 'ttcgagacgcagctcaagac',
 'tcgagacgcagctcaagacc']

In [346]:
#generates list of primer window temperatures
def primer_window_temps(primer):
    primerwindows=["".join(x) for x in window(primer, 20)]
    meltingtemps=[]
    for w in primerwindows:
        AT=w.count('a')+w.count('t')
        GC=w.count('g')+w.count('c')
        temp=(float(AT)*2) + (float(GC)*4)
        meltingtemps.append(temp)
    return meltingtemps

#array of melting temperatures of all windows
#turn into numpy array
nptemps = np.array(primer_window_temps(primer))
nptemps

array([68., 68., 68., 66., 66., 66., 64., 64., 62., 62., 64.])

In [347]:
#Stats of melting temperatures
np.mean(nptemps)

65.27272727272727

In [350]:
np.min(nptemps)

62.0

In [351]:
np.max(nptemps)

68.0

In [ ]:
#range is min-max

## 1.B. Full Code for Part I
Including user input: run entire cell to execute code

In [364]:
###FULL SCRIPT

import numpy as np
import pandas as pd
import os
import re
from statistics import mean
from itertools import islice

print("Input fasta file path: ")
file=input()

#PARSING FASTA

f=open(file)
sequence_names=[]
sequences=[]
count=0
seq=""

for line in f:
    if line.startswith(">"):
        sequence_names.append(line[1:].strip()) #to remove the > and \n
        count=count+1
        sequences.append("") #initializing the slot for the sequence
    elif not line.startswith('#') and line!="":
        sequences[count-1]=sequences[count-1]+line.strip().replace('-',"")

f.close()

#initialize data
data = {'ID':sequence_names,
        'Sequence':sequences}
  
#dataframe
dfseq = pd.DataFrame(data)
#set index as sequence ID
dfseq=dfseq.set_index('ID')

print("The number of sequences in " + file + " is", len(dfseq))

#extracting primers, introns, and exons, and storing then in pandas dataframe
primers=[]
exons=[]
introns=[]

for i in range(0, len(dfseq)):
    p=re.findall('[a-z]+', dfseq.Sequence[i])[0] #the first lowercase is the primer
    primers.append(p)
    
    ex=re.findall('[A-Z]+', dfseq.Sequence[i]) #all uppercases are exons
    exons.append(ex)
    
    intr=re.findall('[a-z]+', dfseq.Sequence[i])[1:-1] #all lowercases are introns: exclude upperstream and lowerstream
    introns.append(intr)
    
#adding primer, exons and introns to the dataframe
dfseq['Primer']=primers
dfseq['Exons']=exons
dfseq['Introns']=introns
#dfseq is a dataframe with seqid, sequence, primer, intron, exon

#FUNCTIONS (to be used with dataframe.apply) to calculate statistics

#function that returns average string length in a list of strings
#uses mean from statistics package
def avgstrlen(list):
    return mean([len(i) for i in list])

#function that calculates the percentage occurence of a letter in a list of strings
def occurencepercentage(letter, list):
    str=''.join(list).upper() #case insensitive, join list elements into a single string
    freq=str.count(letter.upper()) #occurence of letter in string
    perc=100*(float(freq)/float(len(str))) #percentage
    return perc

#functions that calculate percentage occurence of each nucleotide (to be used with apply)
def occurenceA(list):
    return occurencepercentage('A', list)

def occurenceG(list):
    return occurencepercentage('G', list)

def occurenceC(list):
    return occurencepercentage('C', list)

def occurenceT(list):
    return occurencepercentage('T', list)

#ADDING STATS to dfseq
#number of exons
dfseq['NumExons']=dfseq['Exons'].apply(len).tolist()

#number of introns
dfseq['NumIntrons']=dfseq['Introns'].apply(len).tolist()

#AvgExonLength
dfseq['AvgExonLength']=dfseq['Exons'].apply(avgstrlen).tolist()
#AvgIntronLength
dfseq['AvgIntronLength']=dfseq['Introns'].apply(avgstrlen).tolist()

#%A_Exon
dfseq['%A_Exon']=dfseq['Exons'].apply(occurenceA).tolist()
#%C_Exon
dfseq['%C_Exon']=dfseq['Exons'].apply(occurenceC).tolist()
#%G_Exon
dfseq['%G_Exon']=dfseq['Exons'].apply(occurenceG).tolist()
#%T_Exon
dfseq['%T_Exon']=dfseq['Exons'].apply(occurenceT).tolist()

#%A_Intron
dfseq['%A_Intron']=dfseq['Introns'].apply(occurenceA).tolist()
#%C_Intron
dfseq['%C_Intron']=dfseq['Introns'].apply(occurenceC).tolist()
#%G_Intron
dfseq['%G_Intron']=dfseq['Introns'].apply(occurenceG).tolist()
#%T_Intron
dfseq['%T_Intron']=dfseq['Introns'].apply(occurenceT).tolist()

#extracting only needed columns
dfseqstat=dfseq[['NumExons','NumIntrons', 'AvgExonLength', 'AvgIntronLength', '%A_Exon', '%C_Exon', '%G_Exon', '%T_Exon', '%A_Intron', '%C_Intron','%G_Intron','%T_Intron']]

#rounding for cleaner output
dfseqstat=dfseqstat.round()

#Write to DNAStats.txt
filename=file+'DNAStats.txt' #to avoid overwriting for multiple runs but with different filenames
dfseqstat.to_csv(filename, header=True, index=True, sep='\t')
print("The file " +filename+" was created.")
print("")
print(dfseqstat)
print("")

#Retrieving sequence stats from created file
newdf=pd.read_csv(filename, sep='\t', engine='python') #reading the file into a pd dataframe
newdf=newdf.set_index('ID') #setting the ID as the index

#user input sequence ID
idx=newdf.index #the index of the dataframe, contains all sequence IDs
flag = True
while flag:
    print("Input sequence ID to retrieve info: ")
    selection=input()
    if selection not in idx:
        print("sequence ID not found: ")
    else:
        flag=False

info = newdf.loc[selection]
print("")
print("Info for " + selection)
print(info)

#Translation of selection
#getting the exon sequence of the user selection (SeqID)
exons=dfseq.loc[selection,'Exons']

#joining the exons into one exon string
exonseq=''.join(exons).upper()
exonseq=exonseq.replace("T", "U")
print("")
print("MRNA Sequence: " +exonseq)

#function that translates a given mRNA into a protein sequence
def translate(seq):
    protein =""
    
    cmap = {"UUU":"F", "UUC":"F", "UUA":"L", "UUG":"L", "UCU":"S", "UCC":"S", "UCA":"S", 
           "UCG":"S", "UAU":"Y", "UAC":"Y", "UAA":"STOP", "UAG":"STOP", "UGU":"C", "UGC":"C", 
           "UGA":"STOP", "UGG":"W", "CUU":"L", "CUC":"L", "CUA":"L", "CUG":"L", "CCU":"P", 
           "CCC":"P", "CCA":"P", "CCG":"P", "CAU":"H", "CAC":"H", "CAA":"Q", "CAG":"Q", 
           "CGU":"R", "CGC":"R", "CGA":"R", "CGG":"R", "AUU":"I", "AUC":"I", "AUA":"I", 
           "AUG":"M", "ACU":"T", "ACC":"T", "ACA":"T", "ACG":"T", "AAU":"N", "AAC":"N", 
           "AAA":"K", "AAG":"K", "AGU":"S", "AGC":"S", "AGA":"R", "AGG":"R", "GUU":"V", 
           "GUC":"V", "GUA":"V", "GUG":"V", "GCU":"A", "GCC":"A", "GCA":"A", "GCG":"A", 
           "GAU":"D", "GAC":"D", "GAA":"E", "GAG":"E", "GGU":"G", "GGC":"G", "GGA":"G", "GGG":"G"}

    if len(seq)%3 != 0: #if the sequence isn't divisible by 3
        skip=len(seq)%3 #skip the remainder (either last 1 or 2 bases)
    
    for i in range(0, len(seq)-skip, 3):
        codon = seq[i:i+3]
        protein=protein+cmap[codon]
    
    return protein

print("")
print("Peptide Sequence: " +translate(exonseq))

#primer melting temperature: range & average

#getting the primer sequence of the user selection (SeqID)
primer=dfseq.loc[selection,'Primer']

#uses islice
#function that returns a list of sliding window strings
def window(seq, n=20):
    #sliding window of seq of width n
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

primerwindows=["".join(x) for x in window(primer, 20)]

#generates list of primer window temperatures
def primer_window_temps(primer):
    primerwindows=["".join(x) for x in window(primer, 20)]
    meltingtemps=[]
    for w in primerwindows:
        AT=w.count('a')+w.count('t')
        GC=w.count('g')+w.count('c')
        temp=(float(AT)*2) + (float(GC)*4)
        meltingtemps.append(temp)
    return meltingtemps

#array of melting temperatures of all windows
#turn into numpy array
nptemps = np.array(primer_window_temps(primer))

print("")
print("Primer stats: ")
#Stats of melting temperatures
print("Average melting temperature: ", np.mean(nptemps))
print("Minimum melting temperature: ", np.min(nptemps))
print("Maximum melting temperature: ", np.max(nptemps))
print("Range: ", np.max(nptemps)-np.min(nptemps))

FinalProject.fasta
The number of sequences in FinalProject.fasta is 6
The file FinalProject.fastaDNAStats.txt was created.

       NumExons  NumIntrons  AvgExonLength  AvgIntronLength  %A_Exon  %C_Exon  \
ID                                                                              
HCN1          8           7         1242.0          61643.0     30.0     20.0   
HCN2          8           7          428.0           3408.0     15.0     39.0   
KCNA1         2           1         1309.0           4062.0     28.0     24.0   
KCNA2         5           4          392.0           8984.0     24.0     27.0   
KCNS3         3           2          780.0          25976.0     24.0     24.0   
KIR3          3           2          209.0           1668.0     21.0     30.0   

       %G_Exon  %T_Exon  %A_Intron  %C_Intron  %G_Intron  %T_Intron  
ID                                                                   
HCN1      20.0     30.0       32.0       17.0       18.0       33.0  
HCN2      32.0   

# Part II:
## Full Code

In [22]:
#reverse transcription function
import itertools
import numpy as np
import pandas as pd

proteinseq="NFAKSC"

#returns a dataframe with possible sequences and their corresponding GC percentage
def comb_codons(sequence):
    
    #dictionary that maps peptide to possible codons
    codonmap = {"F":["UUU", "UUC"], 
       "L":["UUA","UUG", "CUU", "CUC", "CUA", "CUG"], 
       "S":["UCU", "UUC", "UCA","UCG", "AGU", "AGC"], 
       "Y":["UAU", "UAC"], 
       "STOP":["UAA", "UAG", "UGA"], 
       "C":["UGU","UGC"], 
       "W":["UGG"], 
       "P":["CCU", "CCC","CCA","CCG"],
       "H":["CAU", "CAC"],
       "Q":["CAA","CAG"],
       "R":["CGU", "CGC", "CGA", "CGG", "AGA", "AGG"],
       "I":["AUU", "AUC", "AUA"],
       "M":["AUG"],
       "T":["ACU", "ACC", "ACA", "ACG"],
       "N":["AAU", "AAC"],
       "K":["AAA","AAG"],
       "V":["GUU", "GUC", "GUA", "GUG"],
       "A":["GCU", "GCC", "GCA", "GCG"],
       "D":["GAU","GAC"],
       "E":["GAA", "GAG"],
       "G":["GGU", "GGC", "GGA", "GGG"]
      }

    decoded=[codonmap[letter] for letter in proteinseq] #using a built-in itertools function
    decoded_comb=list(itertools.product(*decoded)) #cartesian product of the sets, returns the possible combinations separated
    #not as a string
    
    combs=[] #stores the possible sequences
    combsGC=[] #stores their GC content
    
    for comb in decoded_comb:
        seq="".join(list(comb)) #joins the sets into one string
        seqGC=100*(float(seq.count("G")+seq.count("C"))/float(len(seq))) #calculate GC content
        combs.append(seq)
        combsGC.append(seqGC)

    dfcomb = pd.DataFrame() 
    dfcomb['PossibleSequence']=combs
    dfcomb['GC_Content']=combsGC
    return dfcomb #dataframe with all the info

df=comb_codons(proteinseq)
print(df.sort_values('GC_Content', ascending=True)) #sorted in ascending order

def findclosest(val, df, col): #finds closest match to a given value in a df in a specific col
    exact = df[df[col] == val]
    if not exact.empty: #if there is an exact match to the value, return the index/eces of this match
        return exact.index
    else:
        lower_ind = df[df[col] < val][col].idxmax()
        upper_ind = df[df[col] > val][col].idxmin()
        return [lower_ind, upper_ind]
    
findclosest(50, df, 'GC_Content')



    

       PossibleSequence  GC_Content
0    AAUUUUGCUAAAUCUUGU   22.222222
56   AAUUUUGCAAAAAGUUGU   22.222222
52   AAUUUUGCAAAAUCAUGU   22.222222
50   AAUUUUGCAAAAUUCUGU   22.222222
8    AAUUUUGCUAAAAGUUGU   22.222222
..                  ...         ...
319  AACUUCGCCAAAUCGUGC   50.000000
331  AACUUCGCCAAGUCGUGC   55.555556
379  AACUUCGCGAAGUCGUGC   55.555556
335  AACUUCGCCAAGAGCUGC   55.555556
383  AACUUCGCGAAGAGCUGC   55.555556

[384 rows x 2 columns]


Int64Index([139, 143, 187, 191, 235, 239, 283, 287, 307, 311, 319, 323, 325,
            327, 329, 330, 333, 334, 355, 359, 367, 371, 373, 375, 377, 378,
            381, 382],
           dtype='int64')